In [1]:
import pandas as pd
import pyodbc
from datamart import fix_me
username = 'vigrose'
password_edw = 'Gryffindor~10946'

In [2]:
ppd = pd.read_csv('../../Data/DataGov/ppd_matches.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [6]:
ppd['ME']=fix_me(ppd.ME)
mes = tuple(ppd.head(50000).ME)

In [9]:
    sql_query = \
        f'''
        SELECT DISTINCT
        A.PARTY_ID,
        V.ME,
        A.POST_CD_ID,
        P.ADDR_1,
        P.ADDR_2,
        P.ADDR_3,
        P.CITY,
        S.SRC_STATE_CD AS STATE_CD,
        P.POST_CD AS ZIP,
        A.FROM_DT,
        CC.DESC,
        CC.CATEGORY_CODE
        FROM
        AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CAT_CD CC, AMAEDW.PARTY_ID_TO_ME_VW V, AMAEDW.CONT_PURPOSE_TYPE C
        WHERE
        A.POST_CD_ID = P.POST_CD_ID
        AND
        P.STATE_ID = S.STATE_ID
        AND
        A.THRU_DT IS NULL
        AND
        CC.CAT_CD_ID = A.CAT_CD_ID
        AND
        V.PARTY_ID = A.PARTY_ID
        AND
        V.ME in {mes}
        AND
        A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
        AND
        C.PURPOSE_USG_CD='PO'
        '''

In [4]:
z = 50000
chunks = [50000]
for x in list(range(15)):
    z+=20000
    print(z)
    chunks.append(z)

70000
90000
110000
130000
150000
170000
190000
210000
230000
250000
270000
290000
310000
330000
350000


In [11]:
for chunk in chunks:
    mes = tuple(ppd[chunk:chunk+20000].ME)
    sql_query = \
        f'''
        SELECT DISTINCT
        A.PARTY_ID,
        V.ME,
        A.POST_CD_ID,
        P.ADDR_1,
        P.ADDR_2,
        P.ADDR_3,
        P.CITY,
        S.SRC_STATE_CD AS STATE_CD,
        P.POST_CD AS ZIP,
        A.FROM_DT,
        C.DESC,
        C.CATEGORY_CODE
        FROM
        AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CAT_CD C, AMAEDW.PARTY_ID_TO_ME_VW V
        WHERE
        A.POST_CD_ID = P.POST_CD_ID
        AND
        P.STATE_ID = S.STATE_ID
        AND
        A.THRU_DT IS NULL
        AND
        C.CAT_CD_ID = A.CAT_CD_ID
        AND
        V.PARTY_ID = A.PARTY_ID
        AND
        V.ME in {mes}
        '''
    new_usages = pd.read_sql(con=AMAEDW, sql=sql_query)
    usages = pd.concat([usages, new_usages])
    print(f'{chunk} finished. {len(new_usages)} added.')

50000 finished. 153299 added.
70000 finished. 153528 added.
90000 finished. 143983 added.
110000 finished. 149409 added.
130000 finished. 142897 added.
150000 finished. 132213 added.
170000 finished. 149470 added.
190000 finished. 142969 added.
210000 finished. 144350 added.
230000 finished. 151692 added.
250000 finished. 148027 added.
270000 finished. 137577 added.
290000 finished. 141013 added.
310000 finished. 138916 added.
330000 finished. 142022 added.
350000 finished. 30653 added.


In [10]:
usages = pd.read_sql(con=AMAEDW, sql=sql_query)

In [25]:
ppd['POLO_MAILING_LINE_2']

0                 14502 W MEEKER BLVD
1               4429 CLARA ST STE 400
2              300 N AIRPORT RD STE 2
3         208 MCFARLAND CIR N STE 100
4                     2880 DAUPHIN ST
                     ...             
354336             3235 N WINDSONG DR
354337                540 W PUEBLO ST
354338            1000 DUTCH RIDGE RD
354339                730 W MARKET ST
354340          1 MEDICAL CENTER BLVD
Name: POLO_MAILING_LINE_2, Length: 354341, dtype: object

In [58]:
usages

,PARTY_ID,ME,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT,DESC,CATEGORY_CODE
0,3458344,00576021372,9447016,2600 Redondo Ave,,,Long Beach,CA,90806,2016-10-17 14:23:06.699740,Group Roster/Census,GROUP
1,1880099,01103930851,12258771,1000 36th St,,,Vero Beach,FL,32960,2016-10-17 14:23:08.084275,Group Roster/Census,GROUP
2,1888761,00102720873,16274554,701 Princeton Ave SW,,,Birmingham,AL,35211,2016-10-17 14:23:08.590263,Group Roster/Census,GROUP
3,1799981,00102951930,16274554,701 Princeton Ave SW,,,Birmingham,AL,35211,2016-10-17 14:23:08.664613,Group Roster/Census,GROUP
4,1798521,00102741030,16274555,PO Box 798,,,Florence,AL,35631,2016-10-17 14:23:08.855170,FAST-TRACK,FAST-TRACK
...,...,...,...,...,...,...,...,...,...,...,...,...
368232,1922774,01401780889,32118357,1029 Kapahulu Ave,Ste 301,,Honolulu,HI,96816,2021-03-15 20:48:26.929312,Membership Join Renew physician address.,JOIN-RENEW
368233,4895962,00519110148,14169661,15 McCabe Dr,# 200,,Reno,NV,89511,2021-03-16 00:04:23.000000,State Licensure,LICENSURE
368234,1817700,00518811045,31954116,774 Mays Blvd,Ste 10,PMB 714,Incline Vlg,NV,89451,2021-03-16 00:04:23.000000,State Licensure,LICENSURE
368235,1817571,00518820648,32119335,PO BOX 183,None,None,DAVIS,CA,95617,2021-03-16 00:04:23.000000,State Licensure,LICENSURE


In [12]:
usages = usages.fillna('None')

In [13]:
usages['POLO_MAILING_LINE_2']=[x.upper() for x in usages.ADDR_1]

In [14]:
sourcing = pd.merge(usages, ppd, on=['ME','POLO_MAILING_LINE_2'])

In [75]:
ppd[ppd.ME.isin(usages.ME)==False]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,NPI


In [28]:
usages['POLO_MAILING_LINE_2']

0       <built-in method upper of str object at 0x0000...
1       <built-in method upper of str object at 0x0000...
2       <built-in method upper of str object at 0x0000...
3       <built-in method upper of str object at 0x0000...
4       <built-in method upper of str object at 0x0000...
                              ...                        
7744    <built-in method upper of str object at 0x0000...
7745    <built-in method upper of str object at 0x0000...
7746    <built-in method upper of str object at 0x0000...
7747    <built-in method upper of str object at 0x0000...
7748    <built-in method upper of str object at 0x0000...
Name: POLO_MAILING_LINE_2, Length: 7749, dtype: object

In [ ]:
'''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE_CD,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL;
    '''

In [52]:
len(ppd)/10000

35.4341

In [60]:
(len(ppd)-50000)/20000

15.21705

In [67]:
z = 50000
chunks = [50000]
for x in list(range(15)):
    z+=20000
    print(z)
    chunks.append(z)

70000
90000
110000
130000
150000
170000
190000
210000
230000
250000
270000
290000
310000
330000
350000


In [68]:
chunks

[50000,
 70000,
 90000,
 110000,
 130000,
 150000,
 170000,
 190000,
 210000,
 230000,
 250000,
 270000,
 290000,
 310000,
 330000,
 350000]

In [69]:
chunks[0:2]

[50000, 70000]

In [71]:
ppd[2:4]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,NPI
2,00102000064,A,1,2.0,CAROL L ADAMS MD,ADAMS,CAROL,LYNN,NaN,NaN,...,35504.0,2516.0,C009,DANIELS,LYNN,CAROL,NaN,NaN,NaN,1083671994
3,00102000072,A,1,2.0,JAMES H BANKSTON JR MD,BANKSTON,JAMES,HAROLD,2.0,NaN,...,35406.0,1800.0,C057,NaN,NaN,NaN,NaN,NaN,NaN,1831130459


In [72]:
len(mes)

50000

In [85]:
sourcing.groupby('DESC').count().sort_values('PARTY_ID', ascending = False)[['PARTY_ID']].to_csv('../../Data/DataGov/Sourcing.csv')

In [89]:
len(sourcing)

999017

In [87]:
ppd[ppd.ME.isin(sourcing.ME)==False]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,NPI
3,00102000072,A,1,2.0,JAMES H BANKSTON JR MD,BANKSTON,JAMES,HAROLD,2.0,NaN,...,35406.0,1800.0,C057,NaN,NaN,NaN,NaN,NaN,NaN,1831130459
9,00102000170,A,1,2.0,MARTIN T SIGLER MD,SIGLER,MARTIN,TRISTRAM,NaN,NaN,...,75093.0,5215.0,C095,NaN,NaN,NaN,NaN,NaN,NaN,1245299692
10,00102000196,A,1,1.0,REBECCA L SUGG MD,SUGG,REBECCA,L M,NaN,USA DEPT OF NEUROLOGY,...,36617.0,2300.0,C011,MIXON,LEIGH,REBECCA,NaN,NaN,NaN,1982656773
19,00102000366,A,1,1.0,ALAN C MC COOL MD,MC COOL,ALAN,CLAYTON,NaN,NaN,...,35501.0,8952.0,C006,NaN,NaN,NaN,NaN,NaN,NaN,1811940679
20,00102000391,A,1,1.0,GREGORY J FOLSOM MD,FOLSOM,GREGORY,JOHN,NaN,NaN,...,55439.0,2569.0,C001,NaN,NaN,NaN,NaN,NaN,NaN,1033184213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354326,96501850036,A,1,1.0,RASHID A DALAL MD,DALAL,RASHID,AHMED M,NaN,NaN,...,62226.0,5369.0,C026,NaN,NaN,NaN,NaN,NaN,NaN,1265434815
354329,96501870011,A,1,2.0,ANTHONY EKWENCHI MD,EKWENCHI,ANTHONY,NaN,NaN,NaN,...,30106.0,2007.0,C006,EKWENCHI,UBA,ANTHONY,NaN,NaN,NaN,1376765180
354332,96501880032,A,1,2.0,INDUMATHI N RAO MD,RAO,INDUMATHI,NATARAJAN,NaN,NaN,...,45429.0,2219.0,C017,NaN,NaN,NaN,NaN,NaN,NaN,1447242045
354334,96501900017,A,1,1.0,MAMLE ANIM MD,ANIM,MAMLE,NaN,NaN,NaN,...,45402.0,2643.0,C053,ONWUDIWE,ANIM,MAMLE,NaN,NaN,NaN,1790741536


In [90]:
999017/287778

3.471484964104275

In [95]:
sourcing[['FROM_DT_x']].dtypes

FROM_DT_x    datetime64[ns]
dtype: object

In [16]:
sourcing.sort_values('FROM_DT_x').drop_duplicates('ME', keep='last').groupby('DESC').count().sort_values('PARTY_ID', ascending = False)[['PARTY_ID']].to_csv('../../Data/DataGov/Sourcing_New.csv')

In [17]:
newest = sourcing.sort_values('FROM_DT_x').drop_duplicates('ME', keep='last')

In [20]:
newest['YEAR'] = [x.year for x in newest.FROM_DT_x]

In [22]:
newest.groupby('YEAR').count()[['PARTY_ID']]

,PARTY_ID
YEAR,
2016,67830
2017,32308
2018,22918
2019,31650
2020,85676
2021,6523


In [23]:
from bs4 import BeautifulSoup
import requests

In [ ]:
med_url = 'https://health.usnews.com/doctors/rebecca-hough-242554'
response = requests.get(med_url)
soup = BeautifulSoup(response.content, 'html.parser')
all_text = soup.text